In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

"Victory Mansions is a building with no windows, located in a run-down area of London, chief city of Airstrip One in Oceania. The building is described as dilapidated, lacking basic amenities, and reflecting the oppressive environment of the dystopian society in which the protagonist, Winston, lives. Inside the building, there are glass doors, a hallway with a smell of boiled cabbage, and old rag mats. The hallway also features a large colored poster of a man's face with a heavy black mustache. Winston's flat is on the seventh floor, with each landing having a poster of Big Brother watching. The view from the roof of Victory Mansions allows the simultaneous sight of all four Ministries, with the Ministry of Truth towering above the building. In Winston's flat, there is a telescreen positioned oddly opposite the window, and a shallow alcove possibly meant for bookshelves. The geography of the room and a book Winston acquired from a junk shop inspired him to take a particular action, des